# Reguliere expressies

Zie https://regex101.com/

Veel van wat we doen bij het schrijven van programma's voor biologie kan worden omschreven als het zoeken naar patronen in strings: DNA-, RNA- en eiwitsequenties zijn slechts strings. Veel deze biologische sequenties analyses kunnen worden beschreven in termen van patronen:

- protein domains
- DNA transcription factor binding motifs
- restriction enzyme cut sites
- degenerate PCR primer sites

Het zijn echter niet alleen sequentiegegevens die interessante patronen kunnen hebben. Andere soorten gegevens waarmee we in de biologie te maken hebben komen bijvoorbeeld voor in de vorm van tekenreeksen in tekstbestanden zoals:

- read mapping locations
- geographical sample coordinates
- taxonomic names
- gene names
- gene accession numbers
- BLAST search results

Het gemeenschappelijke thema bij al deze problemen is dat ze zoeken naar een vast patroon. Maar er zijn veel problemen die we willen oplossen die flexibelere patronen vereisen. 

- given a DNA sequence, what's the length of the poly-A tail?
- given a gene accession name, extract the part between the third character and the underscore
- given a protein sequence, determine if it contains this highly redundant protein domain motif

Omdat dit soort problemen op zoveel verschillende gebieden voorkomen, is er een standaardset tools in Python om ermee om te gaan: reguliere expressies. Reguliere expressies zijn expressies in een apart soort taal. Deze zijn regulier omdat ze in elke programmeer taal hetzelfde zijn. Het is een soort kortschrift. Deze reguliere expressie tools zitten in de library `re`


In [1]:
import re
#dir(re)

De library "re" biedt verschillende methoden om zoekopdracht uit te voeren op tekstdata (een string of een bestand). methodes die veel gebruikt worden zijn 

    re.search()
    re.finditer()
    re.findall()
    re.match()
    re.sub()

We gebruiken verschillende componenten 

* een patroon opgegeven in de reguliere expressie taal
* een string die willen doorzoeken met het opgegeven patroon
* de `re` methode die aangeeft hoe we precies willen zoeken.

## `re.search()`
We beginnen met de eenvoudigste tool voor reguliere expressies. `re.search ()` is een `true / false` functie die bepaalt of een patroon al dan niet ergens in een string voorkomt. Er zijn twee argumenten nodig, beide strings. Het eerste argument is het patroon **waarnaar** gezocht moet worden en het tweede argument is de string **waarin** gezocht moet worden. Zo testen we bijvoorbeeld of een DNA-sequentie een EcoRI-restrictiesite bevat:

In [2]:
dna = "ATCGGGTCCACCAGC"
if re.search(r"GGACC", dna) or re.search(r"GGTCC", dna):
    print("restriction site found!")

restriction site found!


## alternate ( | )
Maar een betere manier is om de variatie op de AvaII-site vast te leggen met een reguliere expressie. Een handig kenmerk van reguliere expressies is *alternate*. Om een aantal verschillende alternatieven weer te geven, schrijven we de alternatieven tussen haakjes`()`, gescheiden door een pipe teken `|`. In het geval van AvaII zijn er twee alternatieven voor het derde base - het kan A of T zijn - dus het patroon ziet er zo uit:


GG (A | T) CC

In re.search () geeft ons dat de code:


In [3]:
dna = "ATCGGGTCCACCAGC"
if re.search(r"GG(A|T)CC", dna):
    print("restriction site found!")

restriction site found!


Het BisI-restrictie-enzym snijdt in een nog groter aantal motieven - het patroon is GCNGC, waarbij N elke base vertegenwoordigt. We kunnen dezelfde afwisselingstechniek gebruiken om dit patroon weer te geven:


GC (A | T | G | C) GC


## blokhaken met toegestane tekens [ ]
Er is echter nog een andere functie voor reguliere expressie waarmee we het patroon beknopter kunnen schrijven. Een paar vierkante haken met een lijst met tekens erin kan elk van deze tekens vertegenwoordigen. Het patroon GC [ATGC] GC komt dus overeen met GCAGC, GCTGC, GCGGC en GCCGC. Hier is een programma dat controleert op de aanwezigheid van een BisI-restrictiesite met behulp van:

In [15]:
dna = "ATCGGGTCCACCAGC"
if re.search(r"GG[ATGC]CC", dna):
    print("restriction site found!")

restriction site found!


## een willekeurig teken:   .

Als we willen dat een teken in een patroon overeenkomt met een willekeurig teken in de invoer, kunnen we een punt of punt gebruiken. Het patroon GC.GC komt bijvoorbeeld overeen met alle vier de mogelijkheden in het bovenstaande BisI-voorbeeld. De punt komt echter ook overeen met elk teken dat geen DNA-basis is, of zelfs maar een letter. Daarom zou het hele patroon ook overeenkomen met GCFGC, GC&GC en GC9GC, wat misschien niet is wat we willen, dus wees voorzichtig bij het gebruik van deze functie.

## een teken dat we niet willen:   [^  ]
Soms is het, in plaats van alle acceptabele tekens op te sommen, gemakkelijker om de tekens op te geven die we niet willen matchen. Een  dakje `^` plaatsen aan het begin van een karaktergroep als deze


[^ XYZ]


zal het negeren en overeenkomen met elk teken dat niet in de groep zit. Het bovenstaande voorbeeld komt overeen met elk ander teken dan X, Y of Z.

## quantifiers ? + * { }

#### ? 
Met de hierboven besproken reguliere expressiefuncties kunnen we variatie in de individuele karakters van patronen beschrijven. Een andere groep kenmerken, **quantifiers**, laten we variatie beschrijven in het aantal keren dat een sectie van een patroon wordt herhaald.

Een vraagteken direct na een teken betekent dat dat teken optioneel is - het kan nul of één keer overeenkomen. Dus in het patroon `GAT?C` is de T optioneel en het patroon komt overeen met **GATC of GAC**. Als we een vraagteken op meer dan één teken willen plaatsen, kunnen we de tekens tussen haakjes groeperen. Bijvoorbeeld in het patroon `GGG(AAA)? TTT` is de groep van drie A's optioneel, dus het patroon komt overeen met **GGGAAATTT of GGGTTT**.

#### +
Een plusteken onmiddellijk na een teken of groep betekent dat het teken of de groep aanwezig moet zijn, maar kan een onbeperkt aantal keren worden herhaald - met andere woorden, het komt *een of meer keren overeen*. Het patroon `GGGA+ TTT` komt bijvoorbeeld overeen met drie G's, gevolgd door een of meer A's, gevolgd door drie T's. Het komt dus overeen met **GGGATTT, GGGAATT, GGGAAATT**, etc. maar niet met GGGTTT.

#### *
Een sterretje direct na een teken of groep betekent dat het teken of de groep optioneel is, maar ook kan worden herhaald. Met andere woorden, het komt *nul of meer keren overeen*. Het patroon `GGGA*TTT` komt bijvoorbeeld overeen met drie G's, gevolgd door nul of meer A's, gevolgd door drie T's. Het komt dus overeen met **GGGTTT, GGGATTT, GGGAATTT, etc**. Het is de meest flexibele qualifier.

#### { }
Als we een bepaald aantal herhalingen willen matchen, kunnen we accolades gebruiken. Het volgen van een personage of groep met een enkel nummer tussen accolades komt exact overeen met dat aantal herhalingen. Het patroon `GA{5}T` komt bijvoorbeeld overeen met **GAAAAAT** (5 A's) maar niet met GAAAAT (4 A's) of GAAAAAAT (6 A's). Door een teken of groep te volgen met een paar cijfers tussen accolades gescheiden door een komma, kunnen we een acceptabel bereik van herhalingen specificeren. Het patroon `GA{2,4}T` betekent bijvoorbeeld G, gevolgd door *tussen 2 en 4* A's, gevolgd door T. Het komt dus overeen met **GAAT, GAAAT en GAAAAT**, maar niet met GAT of GAAAAAT.

Net als bij substrings kunnen we de onder- of bovengrenzen weglaten. Een {3,} komt overeen met drie of meer As en G {, 7} komt overeen met maximaal zeven G's. Samenvattend voorbeeld: Het patroon

        [A-Z][a-z]{2,6}
        
matched met alle strings die tussen de 2 en 6 letters zijn uit het alfabet (gewone en hoofdletters)
       

## positions ^ en '&'
De De laatste set reguliere expressietools waar we naar gaan kijken, vertegenwoordigt helemaal geen tekens, maar eerder posities in de invoerreeks. Het dakje-symbool `^` matched het begin van een string en het dollarteken matched met het einde van een string. Het patroon `^AAA` matched met **AAATTT** maar niet met **GGGAAATTT**. Het patroon `GGG`$ komt overeen met **AAAGGG** maar niet met AAAGGGCCC.

In [19]:
dna = "CGCTCNTAGATGCGCRATGACTGCAYTGC" 
matches = re.search(r"^ACGC", dna) 
print(matches)

None


## combineren van alles
Reguliere expressies worden pas echt krachtig als we gaan combineren. We kunnen quantifiers samen met afwisselingen en tekengroepen gebruiken om zeer flexibele patronen te specificeren. Hier is bijvoorbeeld een complex patroon om eukaryotische messenger-RNA-sequenties van volledige lengte te identificeren:

    ^AUG[AUGC]{30,1000}A{5,10}$ 
    
1. een AUG start codon van het begin van de sequentie: `^AUG`
2. gevolgd door 30 and 1000 bases die A, U, G of C kunnen zijn `[AUGC]{30,1000}`
3. gevolgd door een poly-A tail tussen 5 and 10 bases op het eind van de sequentie `A{5,10}$`

## `re.finditer()`

Tot nu toe hebben we alleen gekeken naar re.search(). re.search zoekt in een string en geeft True of False terug, maar kan geen zoekopdracht doen om alle matches te bepalen. Een methode die dat wel kan is re.finditer(). Het volgende voorbeeld zoekt naar alles wat niet DNA is `[^ACGT]` en stopt al die matches in een lijst genaamd `matches`. Elke match heeft als eigenschap de inhoud van de match (group) en de startpositie van de match (start)

In [5]:
dna = "CGCTCNTAGATGCGCRATGACTGCAYTGC" 

matches = re.finditer(r"[^ATGC]", dna) 
for m in matches: 
    base = m.group() 
    pos  = m.start() 
    print(base + " found at position " + str(pos))

N found at position 5
R found at position 15
Y found at position 25


In [6]:
# het nadeel is wel dat we er door heen moeten loopen. Anders krijgen we het iteratie object terug.
dna = "CGCTCNTAGATGCGCRATGACTGCAYTGC" 
matches = re.finditer(r"[^ATGC]", dna) 
print(matches)

## `re.findall()`
lost het probleem op van het iteratie object.

In [7]:
dna = "CGCTCNTAGATGCGCRATGACTGCAYTGC" 
matches = re.findall(r"[^ATGC]", dna) 
print(matches)

['N', 'R', 'Y']


## `re.match()`
Een laatste methode van de reguliere expressies, is een methode is die lijkt op re.search() genaamd `re.match()`. Het verschil is dat re.search() een patroon identificeert dat *overal* in de string voorkomt, terwijl re.match() alleen een patroon identificeert als het *overeenkomt met de hele string*. 

In [8]:
#een voorbeeld
pat = r"\d{2} [A-Z][a-z]{2,4} [programmeur|ontwikkelaar|systeemmanager]" # # definitie van het patroon
l = ["01 Rose programmeur","32 Piet systeemmanager","42 Marcel ontwikkelaar"] # de strings die we willen doorzoeken          
solution = [substring for substring in l if re.match(pat, substring)] #de methode om te zoeken: match een string tegen het patroon
print(solution)


['01 Rose programmeur', '32 Piet systeemmanager']


In [10]:
#Een voorbeeld
pattern = r"(\w+)\(" # definitie van het patroon
s = "KIAA1279(dist=6336),SRGN(dist=64753)CCSER2(dist=489626)" # de string die we willen doorzoeken                    
runs = re.findall(pattern, s) #de methode om te zoeken: vind alle strings die matchen met het patroon
print(runs)  # het resultaat

['KIAA1279', 'SRGN', 'CCSER2']


## Reguliere expressies in het wild
Soms wordt de reguliere expressie gebruikt in combinatie met replace functies. Dit wordt heel veel gedaan in data cleaning en preprossessing in data science

In [11]:
# een voorbeeld
import pandas as pd
url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/03-11-2020.csv"
df = pd.read_csv(url)
df = df.replace(r'[^\sa-zA-Z0-9]+', "", regex=True) # troep zoals emoi's eruit

## `re.sub()`
In python kunnen we re.sub() gebruiken als we iets willen zoeken en vervangen

In [20]:
dna = 'ACCTTGT--CGAA-GGAGT'
temp_b = re.sub('[\W-]', '', dna)
print(temp_b)


ACCTTGTCGAAGGAGT


## \d \w\ s

Maar wat zijn nu die \d \w \s die we in die reguliere expressie patronen zien? Naast de quantifiers, positions, blokhaken, dakjes en dollartekens hebben we ook nog eens expressies voor groepen van tekens. Eentje waren we er al tegen gekomen: de `.` voor een willekeurig teken. Hier staan ze opgesomd.

    . any character except newline \n
    \d any decimal digit equal to [0-9]
    \D non-digit, equal to [^0-9]
    \s any whitespace character, equal to [...>\t\n\r\f\v]
    \S any non-whitespace character, equal to [^ >\t\n\r\f\v]
    \w any alphanumeric character, equal to [a-zA-Z-0-9_]
    \W any non-alphanumeric character, equal [^a-zA-Z-0-9_]
    

`r'[^\sa-zA-Z0-9]+'` betekent dus elk teken dat niet een spatie is of een letter of een getal

`r"\d{2} [A-Z][a-z]{2,4} [programmeur|ontwikkelaar|systeemmanager]"` betekent een getal van twee nummers, gevolgd door een naam tussen de 2 en 4 letters gevolgd door of wel programmeur, ofwel ontwikkelaar of wel systeemmanager

`r'(\w+)\('` betekent elk alphanumeriek teken van elke lengte gevolgd door een (


## Opdracht
Zin om zelf uit te proberen? Er staat een weekopdracht klaar. Tip: Test je patroon in a test site zoals https://regex101.com/

source: http://pythonforbiologists.com/index.php/introduction-to-python-for-biologists/regular-expressions/